In [79]:
from pyspark import SparkContext, SparkConf
from user_definition import *
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SQLContext
#Do not add additional libraries.


##########
supervisor = sc.textFile(file_name_2).map(lambda x: (x.split(","), len(x.split(",")[0])))
def fun(x):
    if x[1]==5:
        return x[0]
supervisor = supervisor.map(lambda x: fun(x))
business = sc.textFile(file_name_1).map(lambda x: (x.split(","), len(x.split(",")[0])))
business = business.map(lambda x: fun(x))


In [81]:
business.first()

[u'94123', u'Tournahu George L', u'3301 Broderick St', u'San Francisco', u'CA']

In [98]:

supervisorSchema = StructType([
    StructField("zip", IntegerType(), False),
    StructField("supervisor", IntegerType(), True)
])

businessSchema = StructType([
    StructField("zip", IntegerType(), False),
    StructField("business_name", StringType(), True),
    StructField("street", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True)
])


def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def toStrSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def stringToSup(r):
  return (toISafe(r[0]), toIntSafe(r[1]))

def stringToBus(r):
  return (
    toIntSafe(r[0]),
    toStrSafe(r[1]),
    toStrSafe(r[2]),
    toStrSafe(r[3]),
    toStrSafe(r[4])
)

# Create RDD using map function
supervisorRDD = supervisor.map(lambda x: stringToSup(x))
businessRDD = business.map(lambda x: stringToBus(x))

# Transform to DF using the schema
supervisorStructDF = sqlContext.createDataFrame(supervisorRDD, supervisorSchema)
businessStructDF = sqlContext.createDataFrame(businessRDD, businessSchema)

# Join tables
joinedDF = supervisorStructDF.join(businessStructDF, "zip", "inner")

# Remove zips < 5
#zip5 = udf(lambda x: len(x)==5)
#joinedDF.filter(zip5("zip")).show()

#supervisorStructDF.printSchema()
#businessStructDF.printSchema()

# print schema
joinedDF.distinct().printSchema()



root
 |-- zip: integer (nullable = false)
 |-- supervisor: integer (nullable = true)
 |-- business_name: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)



In [96]:
businessStructDF.show(10)

+-----+-------------+------+----+-----+
|  zip|business_name|street|city|state|
+-----+-------------+------+----+-----+
|94123|         null|  null|null| null|
|94124|         null|  null|null| null|
|94105|         null|  null|null| null|
|94108|         null|  null|null| null|
|94107|         null|  null|null| null|
|94109|         null|  null|null| null|
|94102|         null|  null|null| null|
|94102|         null|  null|null| null|
|94109|         null|  null|null| null|
|94107|         null|  null|null| null|
+-----+-------------+------+----+-----+
only showing top 10 rows



In [97]:
supervisorStructDF.join(businessStructDF, "zip", "inner").show()

Py4JJavaError: An error occurred while calling o1788.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 82.0 failed 1 times, most recent failure: Lost task 1.0 in stage 82.0 (TID 93, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/share/spark/spark-2.0.2/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/share/spark/spark-2.0.2/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/share/spark/spark-2.0.2/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-95-a15c2b4058a1>", line 42, in <lambda>
  File "<ipython-input-95-a15c2b4058a1>", line 33, in stringToBus
TypeError: 'NoneType' object has no attribute '__getitem__'

	at org.apache.spark.api.python.Py

In [109]:
from pyspark import SparkContext, SparkConf
from user_definition import *
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SQLContext
#Do not add additional libraries.


##########
def fun(x):
    if x[1]==5:
        return x[0]

supervisor = sc.textFile(file_name_2).map(lambda x: (x.split(","), len(x.split(",")[0])))
supervisor = supervisor.map(lambda x: fun(x))
business = sc.textFile(file_name_1).map(lambda x: (x.split(","), len(x.split(",")[0])))
business = business.map(lambda x: fun(x))

supervisorSchema = StructType([
    StructField("zip", IntegerType(), False),
    StructField("supervisor", IntegerType(), True)
])

businessSchema = StructType([
    StructField("zip", IntegerType(), False),
    StructField("business_name", StringType(), True),
    StructField("street", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True)
])

def stringToSup(r):
  return (int(r[0]), int(r[1]))

def stringToBus(r):
  return (
    int(r[0]),
    r[1],
    r[2],
    r[3],
    r[4]
)

# Create RDD using map function
supervisorRDD = supervisor.map(lambda x: stringToSup(x))
businessRDD = business.map(lambda x: stringToBus(x))

# Transform to DF using the schema
supervisorStructDF = sqlContext.createDataFrame(supervisorRDD, supervisorSchema)
businessStructDF = sqlContext.createDataFrame(businessRDD, businessSchema)

# Join tables
joinedDF = supervisorStructDF.join(businessStructDF, "zip", "inner")

# print schema
joinedDF.distinct().printSchema()

# number of rows of given supervisor_id
#print joinedDF.groupBy("supervisor").count().filter("supervisor = {}".format(supervisor_id)).collect()
print "commented out. Had an error :("

# zip, business_name, city of supervisor_id
#joinedDF.filter("supervisor = {}".format(supervisor_id)).select("zip", "business_name", "city").sort(["zip", "business_name", "city"], ascending = True).show(10)
print "commented out. Had an error :("

root
 |-- zip: integer (nullable = false)
 |-- supervisor: integer (nullable = true)
 |-- business_name: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)

commented out. Had an error :(
commented out. Had an error :(


In [114]:
business.collect()

[[u'94123',
  u'Tournahu George L',
  u'3301 Broderick St',
  u'San Francisco',
  u'CA'],
 [u'94124',
  u'Stephens Institute Inc',
  u'2225 Jerrold Ave',
  u'San Francisco',
  u'CA'],
 [u'94105',
  u'Stephens Institute Inc',
  u'180 New Montgomery St',
  u'San Francisco',
  u'CA'],
 [u'94108',
  u'Stephens Institute Inc',
  u'540 Powell St',
  u'San Francisco',
  u'CA'],
 [u'94107',
  u'Stephens Institute Inc',
  u'460 Townsend St',
  u'San Francisco',
  u'CA'],
 [u'94109',
  u'Stephens Institute Inc',
  u'1835-49 Van Ness Ave',
  u'San Francisco',
  u'CA'],
 [u'94102',
  u'Stephens Institute Inc',
  u'620 Sutter St',
  u'San Francisco',
  u'CA'],
 [u'94102',
  u'Stephens Institute Inc',
  u'655 Sutter St',
  u'San Francisco',
  u'CA'],
 [u'94109',
  u'Stephens Institute Inc',
  u'1055 Pine St',
  u'San Francisco',
  u'CA'],
 [u'94107',
  u'Stephens Institute Inc',
  u'121 Wisconsin St',
  u'San Francisco',
  u'CA'],
 [u'94102',
  u'Stephens Institute Inc',
  u'150 Hayes St',
  u'San F